In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, auc, roc_curve
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
from sklearn.model_selection import GridSearchCV
import pickle

In [6]:
X_train = pd.read_csv('../Resources/Datasets/X_train.csv', index_col = 'building_id')
X_test = pd.read_csv('../Resources/Datasets/X_test.csv', index_col = 'building_id')
y_train = pd.read_csv('../Resources/Datasets/y_train.csv', index_col='building_id')
y_test = pd.read_csv('../Resources/Datasets/y_test.csv', index_col='building_id')

y_train = y_train - 1
y_test = y_test - 1
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()


In [17]:
xgb = XGBClassifier(objective='multi:softmax', num_class=3, n_estimators=100, max_depth=10, learning_rate=0.1, n_jobs=-1, verbosity=0)


xgb.fit(X_train, y_train)
pred = xgb.predict(X_test)


cm = confusion_matrix(y_test, xgb.predict(X_test))
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.69      0.47      0.56      5025
           1       0.73      0.86      0.79     29652
           2       0.76      0.60      0.67     17444

    accuracy                           0.73     52121
   macro avg       0.72      0.64      0.67     52121
weighted avg       0.73      0.73      0.73     52121



In [19]:
params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [4,6,8,10,20],
    'learning_rate': [0.1, 0.01],
    'subsample': [0.5, 0.7, 1]
    
}

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid = GridSearchCV(estimator=xgb, param_grid=params, n_jobs=-1, cv=cv, scoring='accuracy')
grid.fit(X_train, y_train)
best_params = grid.best_params_
print(best_params)
best_score = grid.best_score_
print(best_score)

KeyboardInterrupt: 